**数据查看工具**

可使用 [S3_Browser](http://10.140.0.104:10001/S3_Browser) 查看对象存储上的数据。

**选择Spark集群**

`spark_conf_name` 有8个可选值，分别为 `spark_[1234](_old)?`

集群概况如下，点击集群名字，可查看各集群的当前资源使用情况：

- [spark_1](http://10.140.0.100:8088/cluster/scheduler): 共约30节点，与S集群Ceph有高速网络，CPU主频高，内存高。
- [spark_2](http://10.140.84.107:8088/cluster/scheduler): 共96节点，与P、T集群对象存储有高速网络，CPU核数多，内存少。
- [spark_3](http://10.140.84.31:8088/cluster/scheduler): 共40节点，与P、T集群对象存储有高速网络，CPU核数多，内存少，有常驻任务。
- [spark_4](http://10.140.52.131:8088/cluster/scheduler): 共44节点，与任何对象存储都只有**低速**网络，CPU核数多，内存少，不支持需要Shuffle操作的任务。

其他说明：

- Spark版本默认为较新的 `3.4.x` ，若需使用 `2.4.x` 版本，请添加 `_old` 后缀。
- Spark `2.x` 版本，支持的最高的python版本为 `3.7` ，若选择使用Spark `2.x` ，请切换Notebook的Kernel至 `3.7` 版本。
- 执行 `spark` ，将spark对象输出到Output中，可获得当前Spark Session的UI链接，打开UI可了解作业的执行情况，打开之前请 [配置Hosts](https://aicarrier.feishu.cn/wiki/R4uDwvptPiScaXkJMJvcn8AOnCg) 。
- Spark所能读取的对象存储桶名中，只能包含英文字母，数字和短横线(-)，不能包含下划线(\_)。

**Notebook Kernel的选择**

- 若使用Spark 3.x (不带 `_old` 后缀的集群名)，请使用Pythen 3.10 Kernel。
- 若使用Spark 2.x (即带 `_old` 后缀的集群名)，请使用Pythen 3.7 Kernel。

In [1]:
# create spark session
from pyspark.sql import Row, DataFrame

from app.common.spark_ext import *
from app.common.json_util import *
from app.common.s3 import *

config = {
    "spark_conf_name": "spark_2",
    "skip_success_check": True,
    # 根据个人路径进行替换
    "spark.executorEnv.LLM_WEB_KIT_CFG_PATH": "/share/jiangwenhao/.llm-web-kit.jsonc",
    "spark.dynamicAllocation.maxExecutors": "400",
}

spark = new_spark_session("changeit", config)
spark

/tmp/ipykernel_946/1091275960.py:4: DeprecationWarning: `app.common.spark_ext` is deprecated, use `xinghe.spark` instead.
  from app.common.spark_ext import *
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# read data
# s3://private-cooperate-data/zh-web-baijiahao/ 
# s3://private-crawl-data/zh-web-netease/20241218_p1/ 
# s3://private-crawl-data/zh-web-tencent/20241218_p1/ 
# s3://private-crawl-data/zh-web-sina/20241218_p1/ 
# s3://private-cooperate-data/DouBan/
input_paths = [
    "s3://private-crawl-data/zh-web-netease/20241218_p1/",
]
input_df = read_any_path(spark, ",".join(input_paths), config)

In [5]:
# view data
input_df.printSchema()
input_df.size()

root
 |-- value: string (nullable = true)
 |-- filename: string (nullable = true)



AttributeError: 'DataFrame' object has no attribute 'size'

In [ ]:
import zlib
import base64

def compress_and_decompress_str(input_data: Union[str, bytes], compress: bool = True, base: bool = False) -> Union[str, bytes]:
    compressed_bytes = base64.b64decode(input_data)
    decompressed_bytes = zlib.decompress(compressed_bytes)
    return decompressed_bytes.decode('utf-8')


def uncompress_html(_iter):
    
    for row in _iter:
        d = json_loads(row.value)
        raw_html = d.get("html", "")        
        d.pop("html", None)
        d["html"] = compress_and_decompress_str(raw_html)
        d["dataset_name"] = "cc"
        d["data_source_category"] = "html"                

        yield Row(value=json_dumps(d))


input_map_rdd = input_df.rdd.mapPartitions(uncompress_html)
input_map_rdd.take(1)

In [ ]:
# write data
output_path = "s3://BUCKET/path/to/output/dir/"
write_any_path(output_df, output_path, config)